In [1]:
import pandas as pd
import numpy as np
from keras.models import load_model,model_from_json
import joblib
    

Using TensorFlow backend.


In [6]:
filename_input='esehi.xlsx'
test=pd.read_excel(filename_input)
meal_info=pd.read_csv('meal_info.csv')
fulfilment_center=pd.read_csv('fulfilment_center_info.csv')
test=pd.merge(test,meal_info,left_on='meal_id',right_on='meal_id')
test=pd.merge(test,fulfilment_center,left_on='center_id',right_on='center_id')
categorical=test[['id','category','cuisine','week','city_code','center_type','region_code']]
test.drop(columns=['id','category','cuisine','week','city_code','center_type','region_code'],inplace=True)
test['diff']=test.base_price-test.checkout_price
test[['center_id','meal_id']].astype('int64')


,center_id,meal_id
0,55,1885
1,55,1993
2,55,2539
3,55,2631
4,55,1248
5,55,1778
6,55,1062
7,55,2707
8,55,1207


In [11]:
def fun(r):
    filename=f'{r.center_id:0f}_{r.meal_id:}'
    pipe1=joblib.load('ibm1/'+filename+'1.sav')
    pipe2=joblib.load('ibm1/'+filename+'2.sav')
    r['y1']=pipe1.predict(r)
    r['y2']=pipe2.predict(r)
    file=open('ibm1/'+filename+'.json','r')
    model=file.read()
    file.close()
    lm=model_from_json(model)
    lm.load_weights('ibm1/'+filename+'.h5')
    r['num_orders']=lm.predict[r]
    return r
test=test.apply(fun,axis=1)
pd.concat([test,categorical],axis=1)
test.to_excel(filename_input+".xlsx")

FileNotFoundError: [Errno 2] No such file or directory: '/ibm1/55_18851.sav'

In [4]:
import pandas as pd
df=pd.read_csv('test.csv')
df.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,1028232,146,55,1885,158.11,159.11,0,0
1,1127204,146,55,1993,160.11,159.11,0,0
2,1212707,146,55,2539,157.14,159.14,0,0
3,1082698,146,55,2631,162.02,162.02,0,0
4,1400926,146,55,1248,163.93,163.93,0,0


In [3]:
def fun(r):
    r['y1']=r.id*2
    r['y2']=r.id*3
    r['y3']=r.y1+r.y2
    return r
df.apply(fun,axis=1)

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,y1,y2,y3
0,1028232.0,146.0,55.0,1885.0,158.11,159.11,0.0,0.0,2056464.0,3084696.0,5141160.0
1,1127204.0,146.0,55.0,1993.0,160.11,159.11,0.0,0.0,2254408.0,3381612.0,5636020.0
2,1212707.0,146.0,55.0,2539.0,157.14,159.14,0.0,0.0,2425414.0,3638121.0,6063535.0
3,1082698.0,146.0,55.0,2631.0,162.02,162.02,0.0,0.0,2165396.0,3248094.0,5413490.0
4,1400926.0,146.0,55.0,1248.0,163.93,163.93,0.0,0.0,2801852.0,4202778.0,7004630.0
...,...,...,...,...,...,...,...,...,...,...,...
32568,1250239.0,155.0,61.0,1543.0,482.09,484.09,0.0,0.0,2500478.0,3750717.0,6251195.0
32569,1039516.0,155.0,61.0,2304.0,483.09,483.09,0.0,0.0,2079032.0,3118548.0,5197580.0
32570,1158107.0,155.0,61.0,2664.0,322.07,323.07,0.0,0.0,2316214.0,3474321.0,5790535.0
32571,1444235.0,155.0,61.0,2569.0,322.07,323.07,0.0,0.0,2888470.0,4332705.0,7221175.0
